Colab para generar features (extraido de la ayudita que dio el profe Alejandro en Zullip)

"A diferencia de la competencia 1, ahora vamos a usar una base de datos en disco. Todo va a ser más lento, pero se va a poder trabajar sin problemas en máquinas de menor capacidad. La DB va a queda en el archivo dmeyf.db, lo probé en un colab básico y funcionó sin problemas."


In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:////databases/dmeyf.db

"Cargamos el dataset, y como tenemos la base en el disco, no debemos cargarlo más, la próxima se puede usar directamente"

#Features históricas

Ver todos los campos

In [2]:
# %%sql
# CREATE OR REPLACE TABLE competencia_03 AS FROM read_csv_auto('C:/databases/datasets/competencia_03.csv.gz'); 

In [3]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'competencia_03';

,column_name,data_type
0,numero_de_cliente,BIGINT
1,foto_mes,BIGINT
2,active_quarter,BIGINT
3,cliente_vip,BIGINT
4,internet,BIGINT
...,...,...
150,Visa_mconsumototal,DOUBLE
151,Visa_cconsumos,BIGINT
152,Visa_cadelantosefectivo,BIGINT
153,Visa_mpagominimo,DOUBLE


In [4]:
%%sql
SELECT COUNT(*), clase_ternaria 
FROM competencia_03 
GROUP BY clase_ternaria;

,count_star(),clase_ternaria
0,21329,BAJA+2
1,328943,None
2,22106,BAJA+1
3,4520650,CONTINUA


Calcular el valor del mes anterior de un campo


Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis




In [5]:
%%time
%%sql
create or replace table resultado as
SELECT
  *, 
  CAST(floor(foto_mes/100) AS INTEGER) as foto_ano
FROM competencia_03
WHERE foto_mes != 202006

CPU times: total: 41 s
Wall time: 18.7 s


,Success


In [6]:
campos_lag = [
    'thomebanking',
    'cdescubierto_preacordado', 
    'Visa_status',
    'Visa_delinquency',
    'Master_status',
    'Master_delinquency'
]

campos_lag_delta = [
    'ctrx_quarter',
    'ctarjeta_debito_transacciones', 
    'mpasivos_margen', 
    'ctransferencias_emitidas', 
    'cpayroll_trx', 
    'mautoservicio' , 
    'mcuenta_corriente',
    'mtransferencias_emitidas' , 
    'mpayroll' , 
    'mcomisiones_mantenimiento', 
    'mcomisiones_otras', 
    'mcomisiones', 
    'mcaja_ahorro', 
    'mcuentas_saldo', 
    'mtarjeta_visa_consumo',
    'mprestamos_personales',
    'ctarjeta_visa_transacciones',
    'ctarjeta_master_transacciones',
    'Visa_madelantodolares',
    'Visa_mlimitecompra',
    'Visa_msaldopesos',
    'Visa_mpagospesos',
    'Visa_mpagominimo',
    'Visa_msaldodolares',
    'Visa_msaldototal',
    'Visa_cconsumos',
    'Visa_mconsumospesos',
    'Master_madelantodolares',
    'Master_mlimitecompra',
    'Master_msaldopesos',
    'Master_mpagospesos',
    'Master_mpagominimo',
    'Master_msaldodolares',
    'Master_msaldototal',
    'Master_mconsumospesos',
    'Master_cconsumos'
]
#campos = ['ctrx_quarter', 'mpasivos_margen', 'mautoservicio' , 'ctarjeta_debito_transacciones' , 'mtransferencias_emitidas' , 'ctransferencias_emitidas' , 'mpayroll' , 'cpayroll_trx', 'mcomisiones_mantenimiento', 'mcomisiones_otras', 'mcomisiones', 'mcaja_ahorro', 'mcuentas_saldo', 'mtarjeta_visa_consumo','mprestamos_personales']


### Lags y lag delta variables 6 meses

In [7]:
### Query para 6 meses
nuevos_features = ""
for campo in campos_lag:
  nuevos_features += f", lag({campo}, 1) over ventana as {campo}_lag_1\n"
  nuevos_features += f", lag({campo}, 2) over ventana as {campo}_lag_2\n"
  nuevos_features += f", lag({campo}, 3) over ventana as {campo}_lag_3\n"
  nuevos_features += f", lag({campo}, 4) over ventana as {campo}_lag_4\n"
  nuevos_features += f", lag({campo}, 5) over ventana as {campo}_lag_5\n"
  nuevos_features += f", lag({campo}, 6) over ventana as {campo}_lag_6\n"
  nuevos_features += f", lag({campo}, 7) over ventana as {campo}_lag_7\n"
  nuevos_features += f", lag({campo}, 8) over ventana as {campo}_lag_8\n"

for campo in campos_lag_delta:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana as {campo}_slope\n"
  nuevos_features += f", lag({campo}, 1) over ventana as {campo}_lag_1\n"
  nuevos_features += f", lag({campo}, 2) over ventana as {campo}_lag_2\n"
  nuevos_features += f", lag({campo}, 3) over ventana as {campo}_lag_3\n"
  nuevos_features += f", lag({campo}, 4) over ventana as {campo}_lag_4\n"
  nuevos_features += f", lag({campo}, 5) over ventana as {campo}_lag_5\n"
  nuevos_features += f", lag({campo}, 6) over ventana as {campo}_lag_6\n"
  nuevos_features += f", lag({campo}, 7) over ventana as {campo}_lag_7\n"
  nuevos_features += f", lag({campo}, 8) over ventana as {campo}_lag_8\n"
  nuevos_features += f", {campo} - lag({campo}, 1) over ventana as {campo}_lag_delta_1\n"
  nuevos_features += f", {campo} - lag({campo}, 2) over ventana as {campo}_lag_delta_2\n"
  nuevos_features += f", {campo} - lag({campo}, 3) over ventana as {campo}_lag_delta_3\n"
  nuevos_features += f", {campo} - lag({campo}, 4) over ventana as {campo}_lag_delta_4\n"
  nuevos_features += f", {campo} - lag({campo}, 5) over ventana as {campo}_lag_delta_5\n"
  nuevos_features += f", {campo} - lag({campo}, 6) over ventana as {campo}_lag_delta_6\n"
  nuevos_features += f", {campo} - lag({campo}, 7) over ventana as {campo}_lag_delta_7\n"
  nuevos_features += f", {campo} - lag({campo}, 8) over ventana as {campo}_lag_delta_8\n"
  nuevos_features += f", avg({campo}) over ventana as {campo}_media\n"

print(nuevos_features)

, lag(thomebanking, 1) over ventana as thomebanking_lag_1
, lag(thomebanking, 2) over ventana as thomebanking_lag_2
, lag(thomebanking, 3) over ventana as thomebanking_lag_3
, lag(thomebanking, 4) over ventana as thomebanking_lag_4
, lag(thomebanking, 5) over ventana as thomebanking_lag_5
, lag(thomebanking, 6) over ventana as thomebanking_lag_6
, lag(thomebanking, 7) over ventana as thomebanking_lag_7
, lag(thomebanking, 8) over ventana as thomebanking_lag_8
, lag(cdescubierto_preacordado, 1) over ventana as cdescubierto_preacordado_lag_1
, lag(cdescubierto_preacordado, 2) over ventana as cdescubierto_preacordado_lag_2
, lag(cdescubierto_preacordado, 3) over ventana as cdescubierto_preacordado_lag_3
, lag(cdescubierto_preacordado, 4) over ventana as cdescubierto_preacordado_lag_4
, lag(cdescubierto_preacordado, 5) over ventana as cdescubierto_preacordado_lag_5
, lag(cdescubierto_preacordado, 6) over ventana as cdescubierto_preacordado_lag_6
, lag(cdescubierto_preacordado, 7) over vent

el resultado anterior lo uso para esta nueva query

In [8]:
%%time
%%sql
create or replace table resultado as
SELECT *
, lag(thomebanking, 1) over ventana as thomebanking_lag_1
, lag(thomebanking, 2) over ventana as thomebanking_lag_2
, lag(thomebanking, 3) over ventana as thomebanking_lag_3
, lag(thomebanking, 4) over ventana as thomebanking_lag_4
, lag(thomebanking, 5) over ventana as thomebanking_lag_5
, lag(thomebanking, 6) over ventana as thomebanking_lag_6
, lag(thomebanking, 7) over ventana as thomebanking_lag_7
, lag(thomebanking, 8) over ventana as thomebanking_lag_8
, lag(cdescubierto_preacordado, 1) over ventana as cdescubierto_preacordado_lag_1
, lag(cdescubierto_preacordado, 2) over ventana as cdescubierto_preacordado_lag_2
, lag(cdescubierto_preacordado, 3) over ventana as cdescubierto_preacordado_lag_3
, lag(cdescubierto_preacordado, 4) over ventana as cdescubierto_preacordado_lag_4
, lag(cdescubierto_preacordado, 5) over ventana as cdescubierto_preacordado_lag_5
, lag(cdescubierto_preacordado, 6) over ventana as cdescubierto_preacordado_lag_6
, lag(cdescubierto_preacordado, 7) over ventana as cdescubierto_preacordado_lag_7
, lag(cdescubierto_preacordado, 8) over ventana as cdescubierto_preacordado_lag_8
, lag(Visa_status, 1) over ventana as Visa_status_lag_1
, lag(Visa_status, 2) over ventana as Visa_status_lag_2
, lag(Visa_status, 3) over ventana as Visa_status_lag_3
, lag(Visa_status, 4) over ventana as Visa_status_lag_4
, lag(Visa_status, 5) over ventana as Visa_status_lag_5
, lag(Visa_status, 6) over ventana as Visa_status_lag_6
, lag(Visa_status, 7) over ventana as Visa_status_lag_7
, lag(Visa_status, 8) over ventana as Visa_status_lag_8
, lag(Visa_delinquency, 1) over ventana as Visa_delinquency_lag_1
, lag(Visa_delinquency, 2) over ventana as Visa_delinquency_lag_2
, lag(Visa_delinquency, 3) over ventana as Visa_delinquency_lag_3
, lag(Visa_delinquency, 4) over ventana as Visa_delinquency_lag_4
, lag(Visa_delinquency, 5) over ventana as Visa_delinquency_lag_5
, lag(Visa_delinquency, 6) over ventana as Visa_delinquency_lag_6
, lag(Visa_delinquency, 7) over ventana as Visa_delinquency_lag_7
, lag(Visa_delinquency, 8) over ventana as Visa_delinquency_lag_8
, lag(Master_status, 1) over ventana as Master_status_lag_1
, lag(Master_status, 2) over ventana as Master_status_lag_2
, lag(Master_status, 3) over ventana as Master_status_lag_3
, lag(Master_status, 4) over ventana as Master_status_lag_4
, lag(Master_status, 5) over ventana as Master_status_lag_5
, lag(Master_status, 6) over ventana as Master_status_lag_6
, lag(Master_status, 7) over ventana as Master_status_lag_7
, lag(Master_status, 8) over ventana as Master_status_lag_8
, lag(Master_delinquency, 1) over ventana as Master_delinquency_lag_1
, lag(Master_delinquency, 2) over ventana as Master_delinquency_lag_2
, lag(Master_delinquency, 3) over ventana as Master_delinquency_lag_3
, lag(Master_delinquency, 4) over ventana as Master_delinquency_lag_4
, lag(Master_delinquency, 5) over ventana as Master_delinquency_lag_5
, lag(Master_delinquency, 6) over ventana as Master_delinquency_lag_6
, lag(Master_delinquency, 7) over ventana as Master_delinquency_lag_7
, lag(Master_delinquency, 8) over ventana as Master_delinquency_lag_8

, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana as ctrx_quarter_slope
, lag(ctrx_quarter, 1) over ventana as ctrx_quarter_lag_1
, lag(ctrx_quarter, 2) over ventana as ctrx_quarter_lag_2
, lag(ctrx_quarter, 3) over ventana as ctrx_quarter_lag_3
, lag(ctrx_quarter, 4) over ventana as ctrx_quarter_lag_4
, lag(ctrx_quarter, 5) over ventana as ctrx_quarter_lag_5
, lag(ctrx_quarter, 6) over ventana as ctrx_quarter_lag_6
, lag(ctrx_quarter, 7) over ventana as ctrx_quarter_lag_7
, lag(ctrx_quarter, 8) over ventana as ctrx_quarter_lag_8
, ctrx_quarter - lag(ctrx_quarter, 1) over ventana as ctrx_quarter_lag_delta_1
, ctrx_quarter - lag(ctrx_quarter, 2) over ventana as ctrx_quarter_lag_delta_2
, ctrx_quarter - lag(ctrx_quarter, 3) over ventana as ctrx_quarter_lag_delta_3
, ctrx_quarter - lag(ctrx_quarter, 4) over ventana as ctrx_quarter_lag_delta_4
, ctrx_quarter - lag(ctrx_quarter, 5) over ventana as ctrx_quarter_lag_delta_5
, ctrx_quarter - lag(ctrx_quarter, 6) over ventana as ctrx_quarter_lag_delta_6
, ctrx_quarter - lag(ctrx_quarter, 7) over ventana as ctrx_quarter_lag_delta_7
, ctrx_quarter - lag(ctrx_quarter, 8) over ventana as ctrx_quarter_lag_delta_8
, avg(ctrx_quarter) over ventana as ctrx_quarter_media

, regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana as ctarjeta_debito_transacciones_slope
, lag(ctarjeta_debito_transacciones, 1) over ventana as ctarjeta_debito_transacciones_lag_1
, lag(ctarjeta_debito_transacciones, 2) over ventana as ctarjeta_debito_transacciones_lag_2
, lag(ctarjeta_debito_transacciones, 3) over ventana as ctarjeta_debito_transacciones_lag_3
, lag(ctarjeta_debito_transacciones, 4) over ventana as ctarjeta_debito_transacciones_lag_4
, lag(ctarjeta_debito_transacciones, 5) over ventana as ctarjeta_debito_transacciones_lag_5
, lag(ctarjeta_debito_transacciones, 6) over ventana as ctarjeta_debito_transacciones_lag_6
, lag(ctarjeta_debito_transacciones, 7) over ventana as ctarjeta_debito_transacciones_lag_7
, lag(ctarjeta_debito_transacciones, 8) over ventana as ctarjeta_debito_transacciones_lag_8
, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 1) over ventana as ctarjeta_debito_transacciones_lag_delta_1
, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 2) over ventana as ctarjeta_debito_transacciones_lag_delta_2
, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 3) over ventana as ctarjeta_debito_transacciones_lag_delta_3
, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 4) over ventana as ctarjeta_debito_transacciones_lag_delta_4
, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 5) over ventana as ctarjeta_debito_transacciones_lag_delta_5
, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 6) over ventana as ctarjeta_debito_transacciones_lag_delta_6
, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 7) over ventana as ctarjeta_debito_transacciones_lag_delta_7
, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 8) over ventana as ctarjeta_debito_transacciones_lag_delta_8
, avg(ctarjeta_debito_transacciones) over ventana as ctarjeta_debito_transacciones_media

, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana as mpasivos_margen_slope
, lag(mpasivos_margen, 1) over ventana as mpasivos_margen_lag_1
, lag(mpasivos_margen, 2) over ventana as mpasivos_margen_lag_2
, lag(mpasivos_margen, 3) over ventana as mpasivos_margen_lag_3
, lag(mpasivos_margen, 4) over ventana as mpasivos_margen_lag_4
, lag(mpasivos_margen, 5) over ventana as mpasivos_margen_lag_5
, lag(mpasivos_margen, 6) over ventana as mpasivos_margen_lag_6
, lag(mpasivos_margen, 7) over ventana as mpasivos_margen_lag_7
, lag(mpasivos_margen, 8) over ventana as mpasivos_margen_lag_8
, mpasivos_margen - lag(mpasivos_margen, 1) over ventana as mpasivos_margen_lag_delta_1
, mpasivos_margen - lag(mpasivos_margen, 2) over ventana as mpasivos_margen_lag_delta_2
, mpasivos_margen - lag(mpasivos_margen, 3) over ventana as mpasivos_margen_lag_delta_3
, mpasivos_margen - lag(mpasivos_margen, 4) over ventana as mpasivos_margen_lag_delta_4
, mpasivos_margen - lag(mpasivos_margen, 5) over ventana as mpasivos_margen_lag_delta_5
, mpasivos_margen - lag(mpasivos_margen, 6) over ventana as mpasivos_margen_lag_delta_6
, mpasivos_margen - lag(mpasivos_margen, 7) over ventana as mpasivos_margen_lag_delta_7
, mpasivos_margen - lag(mpasivos_margen, 8) over ventana as mpasivos_margen_lag_delta_8
, avg(mpasivos_margen) over ventana as mpasivos_margen_media

, regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana as ctransferencias_emitidas_slope
, lag(ctransferencias_emitidas, 1) over ventana as ctransferencias_emitidas_lag_1
, lag(ctransferencias_emitidas, 2) over ventana as ctransferencias_emitidas_lag_2
, lag(ctransferencias_emitidas, 3) over ventana as ctransferencias_emitidas_lag_3
, lag(ctransferencias_emitidas, 4) over ventana as ctransferencias_emitidas_lag_4
, lag(ctransferencias_emitidas, 5) over ventana as ctransferencias_emitidas_lag_5
, lag(ctransferencias_emitidas, 6) over ventana as ctransferencias_emitidas_lag_6
, lag(ctransferencias_emitidas, 7) over ventana as ctransferencias_emitidas_lag_7
, lag(ctransferencias_emitidas, 8) over ventana as ctransferencias_emitidas_lag_8
, ctransferencias_emitidas - lag(ctransferencias_emitidas, 1) over ventana as ctransferencias_emitidas_lag_delta_1
, ctransferencias_emitidas - lag(ctransferencias_emitidas, 2) over ventana as ctransferencias_emitidas_lag_delta_2
, ctransferencias_emitidas - lag(ctransferencias_emitidas, 3) over ventana as ctransferencias_emitidas_lag_delta_3
, ctransferencias_emitidas - lag(ctransferencias_emitidas, 4) over ventana as ctransferencias_emitidas_lag_delta_4
, ctransferencias_emitidas - lag(ctransferencias_emitidas, 5) over ventana as ctransferencias_emitidas_lag_delta_5
, ctransferencias_emitidas - lag(ctransferencias_emitidas, 6) over ventana as ctransferencias_emitidas_lag_delta_6
, ctransferencias_emitidas - lag(ctransferencias_emitidas, 7) over ventana as ctransferencias_emitidas_lag_delta_7
, ctransferencias_emitidas - lag(ctransferencias_emitidas, 8) over ventana as ctransferencias_emitidas_lag_delta_8
, avg(ctransferencias_emitidas) over ventana as ctransferencias_emitidas_media

, regr_slope(cpayroll_trx, cliente_antiguedad) over ventana as cpayroll_trx_slope
, lag(cpayroll_trx, 1) over ventana as cpayroll_trx_lag_1
, lag(cpayroll_trx, 2) over ventana as cpayroll_trx_lag_2
, lag(cpayroll_trx, 3) over ventana as cpayroll_trx_lag_3
, lag(cpayroll_trx, 4) over ventana as cpayroll_trx_lag_4
, lag(cpayroll_trx, 5) over ventana as cpayroll_trx_lag_5
, lag(cpayroll_trx, 6) over ventana as cpayroll_trx_lag_6
, lag(cpayroll_trx, 7) over ventana as cpayroll_trx_lag_7
, lag(cpayroll_trx, 8) over ventana as cpayroll_trx_lag_8
, cpayroll_trx - lag(cpayroll_trx, 1) over ventana as cpayroll_trx_lag_delta_1
, cpayroll_trx - lag(cpayroll_trx, 2) over ventana as cpayroll_trx_lag_delta_2
, cpayroll_trx - lag(cpayroll_trx, 3) over ventana as cpayroll_trx_lag_delta_3
, cpayroll_trx - lag(cpayroll_trx, 4) over ventana as cpayroll_trx_lag_delta_4
, cpayroll_trx - lag(cpayroll_trx, 5) over ventana as cpayroll_trx_lag_delta_5
, cpayroll_trx - lag(cpayroll_trx, 6) over ventana as cpayroll_trx_lag_delta_6
, cpayroll_trx - lag(cpayroll_trx, 7) over ventana as cpayroll_trx_lag_delta_7
, cpayroll_trx - lag(cpayroll_trx, 8) over ventana as cpayroll_trx_lag_delta_8
, avg(cpayroll_trx) over ventana as cpayroll_trx_media

, regr_slope(mautoservicio, cliente_antiguedad) over ventana as mautoservicio_slope
, lag(mautoservicio, 1) over ventana as mautoservicio_lag_1
, lag(mautoservicio, 2) over ventana as mautoservicio_lag_2
, lag(mautoservicio, 3) over ventana as mautoservicio_lag_3
, lag(mautoservicio, 4) over ventana as mautoservicio_lag_4
, lag(mautoservicio, 5) over ventana as mautoservicio_lag_5
, lag(mautoservicio, 6) over ventana as mautoservicio_lag_6
, lag(mautoservicio, 7) over ventana as mautoservicio_lag_7
, lag(mautoservicio, 8) over ventana as mautoservicio_lag_8
, mautoservicio - lag(mautoservicio, 1) over ventana as mautoservicio_lag_delta_1
, mautoservicio - lag(mautoservicio, 2) over ventana as mautoservicio_lag_delta_2
, mautoservicio - lag(mautoservicio, 3) over ventana as mautoservicio_lag_delta_3
, mautoservicio - lag(mautoservicio, 4) over ventana as mautoservicio_lag_delta_4
, mautoservicio - lag(mautoservicio, 5) over ventana as mautoservicio_lag_delta_5
, mautoservicio - lag(mautoservicio, 6) over ventana as mautoservicio_lag_delta_6
, mautoservicio - lag(mautoservicio, 7) over ventana as mautoservicio_lag_delta_7
, mautoservicio - lag(mautoservicio, 8) over ventana as mautoservicio_lag_delta_8
, avg(mautoservicio) over ventana as mautoservicio_media

, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana as mcuenta_corriente_slope
, lag(mcuenta_corriente, 1) over ventana as mcuenta_corriente_lag_1
, lag(mcuenta_corriente, 2) over ventana as mcuenta_corriente_lag_2
, lag(mcuenta_corriente, 3) over ventana as mcuenta_corriente_lag_3
, lag(mcuenta_corriente, 4) over ventana as mcuenta_corriente_lag_4
, lag(mcuenta_corriente, 5) over ventana as mcuenta_corriente_lag_5
, lag(mcuenta_corriente, 6) over ventana as mcuenta_corriente_lag_6
, lag(mcuenta_corriente, 7) over ventana as mcuenta_corriente_lag_7
, lag(mcuenta_corriente, 8) over ventana as mcuenta_corriente_lag_8
, mcuenta_corriente - lag(mcuenta_corriente, 1) over ventana as mcuenta_corriente_lag_delta_1
, mcuenta_corriente - lag(mcuenta_corriente, 2) over ventana as mcuenta_corriente_lag_delta_2
, mcuenta_corriente - lag(mcuenta_corriente, 3) over ventana as mcuenta_corriente_lag_delta_3
, mcuenta_corriente - lag(mcuenta_corriente, 4) over ventana as mcuenta_corriente_lag_delta_4
, mcuenta_corriente - lag(mcuenta_corriente, 5) over ventana as mcuenta_corriente_lag_delta_5
, mcuenta_corriente - lag(mcuenta_corriente, 6) over ventana as mcuenta_corriente_lag_delta_6
, mcuenta_corriente - lag(mcuenta_corriente, 7) over ventana as mcuenta_corriente_lag_delta_7
, mcuenta_corriente - lag(mcuenta_corriente, 8) over ventana as mcuenta_corriente_lag_delta_8
, avg(mcuenta_corriente) over ventana as mcuenta_corriente_media

, regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana as mtransferencias_emitidas_slope
, lag(mtransferencias_emitidas, 1) over ventana as mtransferencias_emitidas_lag_1
, lag(mtransferencias_emitidas, 2) over ventana as mtransferencias_emitidas_lag_2
, lag(mtransferencias_emitidas, 3) over ventana as mtransferencias_emitidas_lag_3
, lag(mtransferencias_emitidas, 4) over ventana as mtransferencias_emitidas_lag_4
, lag(mtransferencias_emitidas, 5) over ventana as mtransferencias_emitidas_lag_5
, lag(mtransferencias_emitidas, 6) over ventana as mtransferencias_emitidas_lag_6
, lag(mtransferencias_emitidas, 7) over ventana as mtransferencias_emitidas_lag_7
, lag(mtransferencias_emitidas, 8) over ventana as mtransferencias_emitidas_lag_8
, mtransferencias_emitidas - lag(mtransferencias_emitidas, 1) over ventana as mtransferencias_emitidas_lag_delta_1
, mtransferencias_emitidas - lag(mtransferencias_emitidas, 2) over ventana as mtransferencias_emitidas_lag_delta_2
, mtransferencias_emitidas - lag(mtransferencias_emitidas, 3) over ventana as mtransferencias_emitidas_lag_delta_3
, mtransferencias_emitidas - lag(mtransferencias_emitidas, 4) over ventana as mtransferencias_emitidas_lag_delta_4
, mtransferencias_emitidas - lag(mtransferencias_emitidas, 5) over ventana as mtransferencias_emitidas_lag_delta_5
, mtransferencias_emitidas - lag(mtransferencias_emitidas, 6) over ventana as mtransferencias_emitidas_lag_delta_6
, mtransferencias_emitidas - lag(mtransferencias_emitidas, 7) over ventana as mtransferencias_emitidas_lag_delta_7
, mtransferencias_emitidas - lag(mtransferencias_emitidas, 8) over ventana as mtransferencias_emitidas_lag_delta_8
, avg(mtransferencias_emitidas) over ventana as mtransferencias_emitidas_media

, regr_slope(mpayroll, cliente_antiguedad) over ventana as mpayroll_slope
, lag(mpayroll, 1) over ventana as mpayroll_lag_1
, lag(mpayroll, 2) over ventana as mpayroll_lag_2
, lag(mpayroll, 3) over ventana as mpayroll_lag_3
, lag(mpayroll, 4) over ventana as mpayroll_lag_4
, lag(mpayroll, 5) over ventana as mpayroll_lag_5
, lag(mpayroll, 6) over ventana as mpayroll_lag_6
, lag(mpayroll, 7) over ventana as mpayroll_lag_7
, lag(mpayroll, 8) over ventana as mpayroll_lag_8
, mpayroll - lag(mpayroll, 1) over ventana as mpayroll_lag_delta_1
, mpayroll - lag(mpayroll, 2) over ventana as mpayroll_lag_delta_2
, mpayroll - lag(mpayroll, 3) over ventana as mpayroll_lag_delta_3
, mpayroll - lag(mpayroll, 4) over ventana as mpayroll_lag_delta_4
, mpayroll - lag(mpayroll, 5) over ventana as mpayroll_lag_delta_5
, mpayroll - lag(mpayroll, 6) over ventana as mpayroll_lag_delta_6
, mpayroll - lag(mpayroll, 7) over ventana as mpayroll_lag_delta_7
, mpayroll - lag(mpayroll, 8) over ventana as mpayroll_lag_delta_8
, avg(mpayroll) over ventana as mpayroll_media

, regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana as mcomisiones_mantenimiento_slope
, lag(mcomisiones_mantenimiento, 1) over ventana as mcomisiones_mantenimiento_lag_1
, lag(mcomisiones_mantenimiento, 2) over ventana as mcomisiones_mantenimiento_lag_2
, lag(mcomisiones_mantenimiento, 3) over ventana as mcomisiones_mantenimiento_lag_3
, lag(mcomisiones_mantenimiento, 4) over ventana as mcomisiones_mantenimiento_lag_4
, lag(mcomisiones_mantenimiento, 5) over ventana as mcomisiones_mantenimiento_lag_5
, lag(mcomisiones_mantenimiento, 6) over ventana as mcomisiones_mantenimiento_lag_6
, lag(mcomisiones_mantenimiento, 7) over ventana as mcomisiones_mantenimiento_lag_7
, lag(mcomisiones_mantenimiento, 8) over ventana as mcomisiones_mantenimiento_lag_8
, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 1) over ventana as mcomisiones_mantenimiento_lag_delta_1
, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 2) over ventana as mcomisiones_mantenimiento_lag_delta_2
, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 3) over ventana as mcomisiones_mantenimiento_lag_delta_3
, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 4) over ventana as mcomisiones_mantenimiento_lag_delta_4
, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 5) over ventana as mcomisiones_mantenimiento_lag_delta_5
, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 6) over ventana as mcomisiones_mantenimiento_lag_delta_6
, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 7) over ventana as mcomisiones_mantenimiento_lag_delta_7
, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 8) over ventana as mcomisiones_mantenimiento_lag_delta_8
, avg(mcomisiones_mantenimiento) over ventana as mcomisiones_mantenimiento_media

, regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana as mcomisiones_otras_slope
, lag(mcomisiones_otras, 1) over ventana as mcomisiones_otras_lag_1
, lag(mcomisiones_otras, 2) over ventana as mcomisiones_otras_lag_2
, lag(mcomisiones_otras, 3) over ventana as mcomisiones_otras_lag_3
, lag(mcomisiones_otras, 4) over ventana as mcomisiones_otras_lag_4
, lag(mcomisiones_otras, 5) over ventana as mcomisiones_otras_lag_5
, lag(mcomisiones_otras, 6) over ventana as mcomisiones_otras_lag_6
, lag(mcomisiones_otras, 7) over ventana as mcomisiones_otras_lag_7
, lag(mcomisiones_otras, 8) over ventana as mcomisiones_otras_lag_8
, mcomisiones_otras - lag(mcomisiones_otras, 1) over ventana as mcomisiones_otras_lag_delta_1
, mcomisiones_otras - lag(mcomisiones_otras, 2) over ventana as mcomisiones_otras_lag_delta_2
, mcomisiones_otras - lag(mcomisiones_otras, 3) over ventana as mcomisiones_otras_lag_delta_3
, mcomisiones_otras - lag(mcomisiones_otras, 4) over ventana as mcomisiones_otras_lag_delta_4
, mcomisiones_otras - lag(mcomisiones_otras, 5) over ventana as mcomisiones_otras_lag_delta_5
, mcomisiones_otras - lag(mcomisiones_otras, 6) over ventana as mcomisiones_otras_lag_delta_6
, mcomisiones_otras - lag(mcomisiones_otras, 7) over ventana as mcomisiones_otras_lag_delta_7
, mcomisiones_otras - lag(mcomisiones_otras, 8) over ventana as mcomisiones_otras_lag_delta_8
, avg(mcomisiones_otras) over ventana as mcomisiones_otras_media

, regr_slope(mcomisiones, cliente_antiguedad) over ventana as mcomisiones_slope
, lag(mcomisiones, 1) over ventana as mcomisiones_lag_1
, lag(mcomisiones, 2) over ventana as mcomisiones_lag_2
, lag(mcomisiones, 3) over ventana as mcomisiones_lag_3
, lag(mcomisiones, 4) over ventana as mcomisiones_lag_4
, lag(mcomisiones, 5) over ventana as mcomisiones_lag_5
, lag(mcomisiones, 6) over ventana as mcomisiones_lag_6
, lag(mcomisiones, 7) over ventana as mcomisiones_lag_7
, lag(mcomisiones, 8) over ventana as mcomisiones_lag_8
, mcomisiones - lag(mcomisiones, 1) over ventana as mcomisiones_lag_delta_1
, mcomisiones - lag(mcomisiones, 2) over ventana as mcomisiones_lag_delta_2
, mcomisiones - lag(mcomisiones, 3) over ventana as mcomisiones_lag_delta_3
, mcomisiones - lag(mcomisiones, 4) over ventana as mcomisiones_lag_delta_4
, mcomisiones - lag(mcomisiones, 5) over ventana as mcomisiones_lag_delta_5
, mcomisiones - lag(mcomisiones, 6) over ventana as mcomisiones_lag_delta_6
, mcomisiones - lag(mcomisiones, 7) over ventana as mcomisiones_lag_delta_7
, mcomisiones - lag(mcomisiones, 8) over ventana as mcomisiones_lag_delta_8
, avg(mcomisiones) over ventana as mcomisiones_media

, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana as mcaja_ahorro_slope
, lag(mcaja_ahorro, 1) over ventana as mcaja_ahorro_lag_1
, lag(mcaja_ahorro, 2) over ventana as mcaja_ahorro_lag_2
, lag(mcaja_ahorro, 3) over ventana as mcaja_ahorro_lag_3
, lag(mcaja_ahorro, 4) over ventana as mcaja_ahorro_lag_4
, lag(mcaja_ahorro, 5) over ventana as mcaja_ahorro_lag_5
, lag(mcaja_ahorro, 6) over ventana as mcaja_ahorro_lag_6
, lag(mcaja_ahorro, 7) over ventana as mcaja_ahorro_lag_7
, lag(mcaja_ahorro, 8) over ventana as mcaja_ahorro_lag_8
, mcaja_ahorro - lag(mcaja_ahorro, 1) over ventana as mcaja_ahorro_lag_delta_1
, mcaja_ahorro - lag(mcaja_ahorro, 2) over ventana as mcaja_ahorro_lag_delta_2
, mcaja_ahorro - lag(mcaja_ahorro, 3) over ventana as mcaja_ahorro_lag_delta_3
, mcaja_ahorro - lag(mcaja_ahorro, 4) over ventana as mcaja_ahorro_lag_delta_4
, mcaja_ahorro - lag(mcaja_ahorro, 5) over ventana as mcaja_ahorro_lag_delta_5
, mcaja_ahorro - lag(mcaja_ahorro, 6) over ventana as mcaja_ahorro_lag_delta_6
, mcaja_ahorro - lag(mcaja_ahorro, 7) over ventana as mcaja_ahorro_lag_delta_7
, mcaja_ahorro - lag(mcaja_ahorro, 8) over ventana as mcaja_ahorro_lag_delta_8
, avg(mcaja_ahorro) over ventana as mcaja_ahorro_media

, regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana as mcuentas_saldo_slope
, lag(mcuentas_saldo, 1) over ventana as mcuentas_saldo_lag_1
, lag(mcuentas_saldo, 2) over ventana as mcuentas_saldo_lag_2
, lag(mcuentas_saldo, 3) over ventana as mcuentas_saldo_lag_3
, lag(mcuentas_saldo, 4) over ventana as mcuentas_saldo_lag_4
, lag(mcuentas_saldo, 5) over ventana as mcuentas_saldo_lag_5
, lag(mcuentas_saldo, 6) over ventana as mcuentas_saldo_lag_6
, lag(mcuentas_saldo, 7) over ventana as mcuentas_saldo_lag_7
, lag(mcuentas_saldo, 8) over ventana as mcuentas_saldo_lag_8
, mcuentas_saldo - lag(mcuentas_saldo, 1) over ventana as mcuentas_saldo_lag_delta_1
, mcuentas_saldo - lag(mcuentas_saldo, 2) over ventana as mcuentas_saldo_lag_delta_2
, mcuentas_saldo - lag(mcuentas_saldo, 3) over ventana as mcuentas_saldo_lag_delta_3
, mcuentas_saldo - lag(mcuentas_saldo, 4) over ventana as mcuentas_saldo_lag_delta_4
, mcuentas_saldo - lag(mcuentas_saldo, 5) over ventana as mcuentas_saldo_lag_delta_5
, mcuentas_saldo - lag(mcuentas_saldo, 6) over ventana as mcuentas_saldo_lag_delta_6
, mcuentas_saldo - lag(mcuentas_saldo, 7) over ventana as mcuentas_saldo_lag_delta_7
, mcuentas_saldo - lag(mcuentas_saldo, 8) over ventana as mcuentas_saldo_lag_delta_8
, avg(mcuentas_saldo) over ventana as mcuentas_saldo_media

, regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana as mtarjeta_visa_consumo_slope
, lag(mtarjeta_visa_consumo, 1) over ventana as mtarjeta_visa_consumo_lag_1
, lag(mtarjeta_visa_consumo, 2) over ventana as mtarjeta_visa_consumo_lag_2
, lag(mtarjeta_visa_consumo, 3) over ventana as mtarjeta_visa_consumo_lag_3
, lag(mtarjeta_visa_consumo, 4) over ventana as mtarjeta_visa_consumo_lag_4
, lag(mtarjeta_visa_consumo, 5) over ventana as mtarjeta_visa_consumo_lag_5
, lag(mtarjeta_visa_consumo, 6) over ventana as mtarjeta_visa_consumo_lag_6
, lag(mtarjeta_visa_consumo, 7) over ventana as mtarjeta_visa_consumo_lag_7
, lag(mtarjeta_visa_consumo, 8) over ventana as mtarjeta_visa_consumo_lag_8
, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 1) over ventana as mtarjeta_visa_consumo_lag_delta_1
, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 2) over ventana as mtarjeta_visa_consumo_lag_delta_2
, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 3) over ventana as mtarjeta_visa_consumo_lag_delta_3
, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 4) over ventana as mtarjeta_visa_consumo_lag_delta_4
, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 5) over ventana as mtarjeta_visa_consumo_lag_delta_5
, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 6) over ventana as mtarjeta_visa_consumo_lag_delta_6
, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 7) over ventana as mtarjeta_visa_consumo_lag_delta_7
, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 8) over ventana as mtarjeta_visa_consumo_lag_delta_8
, avg(mtarjeta_visa_consumo) over ventana as mtarjeta_visa_consumo_media

, regr_slope(mprestamos_personales, cliente_antiguedad) over ventana as mprestamos_personales_slope
, lag(mprestamos_personales, 1) over ventana as mprestamos_personales_lag_1
, lag(mprestamos_personales, 2) over ventana as mprestamos_personales_lag_2
, lag(mprestamos_personales, 3) over ventana as mprestamos_personales_lag_3
, lag(mprestamos_personales, 4) over ventana as mprestamos_personales_lag_4
, lag(mprestamos_personales, 5) over ventana as mprestamos_personales_lag_5
, lag(mprestamos_personales, 6) over ventana as mprestamos_personales_lag_6
, lag(mprestamos_personales, 7) over ventana as mprestamos_personales_lag_7
, lag(mprestamos_personales, 8) over ventana as mprestamos_personales_lag_8
, mprestamos_personales - lag(mprestamos_personales, 1) over ventana as mprestamos_personales_lag_delta_1
, mprestamos_personales - lag(mprestamos_personales, 2) over ventana as mprestamos_personales_lag_delta_2
, mprestamos_personales - lag(mprestamos_personales, 3) over ventana as mprestamos_personales_lag_delta_3
, mprestamos_personales - lag(mprestamos_personales, 4) over ventana as mprestamos_personales_lag_delta_4
, mprestamos_personales - lag(mprestamos_personales, 5) over ventana as mprestamos_personales_lag_delta_5
, mprestamos_personales - lag(mprestamos_personales, 6) over ventana as mprestamos_personales_lag_delta_6
, mprestamos_personales - lag(mprestamos_personales, 7) over ventana as mprestamos_personales_lag_delta_7
, mprestamos_personales - lag(mprestamos_personales, 8) over ventana as mprestamos_personales_lag_delta_8
, avg(mprestamos_personales) over ventana as mprestamos_personales_media

, regr_slope(ctarjeta_visa_transacciones, cliente_antiguedad) over ventana as ctarjeta_visa_transacciones_slope
, lag(ctarjeta_visa_transacciones, 1) over ventana as ctarjeta_visa_transacciones_lag_1
, lag(ctarjeta_visa_transacciones, 2) over ventana as ctarjeta_visa_transacciones_lag_2
, lag(ctarjeta_visa_transacciones, 3) over ventana as ctarjeta_visa_transacciones_lag_3
, lag(ctarjeta_visa_transacciones, 4) over ventana as ctarjeta_visa_transacciones_lag_4
, lag(ctarjeta_visa_transacciones, 5) over ventana as ctarjeta_visa_transacciones_lag_5
, lag(ctarjeta_visa_transacciones, 6) over ventana as ctarjeta_visa_transacciones_lag_6
, lag(ctarjeta_visa_transacciones, 7) over ventana as ctarjeta_visa_transacciones_lag_7
, lag(ctarjeta_visa_transacciones, 8) over ventana as ctarjeta_visa_transacciones_lag_8
, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 1) over ventana as ctarjeta_visa_transacciones_lag_delta_1
, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 2) over ventana as ctarjeta_visa_transacciones_lag_delta_2
, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 3) over ventana as ctarjeta_visa_transacciones_lag_delta_3
, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 4) over ventana as ctarjeta_visa_transacciones_lag_delta_4
, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 5) over ventana as ctarjeta_visa_transacciones_lag_delta_5
, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 6) over ventana as ctarjeta_visa_transacciones_lag_delta_6
, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 7) over ventana as ctarjeta_visa_transacciones_lag_delta_7
, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 8) over ventana as ctarjeta_visa_transacciones_lag_delta_8
, avg(ctarjeta_visa_transacciones) over ventana as ctarjeta_visa_transacciones_media

, regr_slope(ctarjeta_master_transacciones, cliente_antiguedad) over ventana as ctarjeta_master_transacciones_slope
, lag(ctarjeta_master_transacciones, 1) over ventana as ctarjeta_master_transacciones_lag_1
, lag(ctarjeta_master_transacciones, 2) over ventana as ctarjeta_master_transacciones_lag_2
, lag(ctarjeta_master_transacciones, 3) over ventana as ctarjeta_master_transacciones_lag_3
, lag(ctarjeta_master_transacciones, 4) over ventana as ctarjeta_master_transacciones_lag_4
, lag(ctarjeta_master_transacciones, 5) over ventana as ctarjeta_master_transacciones_lag_5
, lag(ctarjeta_master_transacciones, 6) over ventana as ctarjeta_master_transacciones_lag_6
, lag(ctarjeta_master_transacciones, 7) over ventana as ctarjeta_master_transacciones_lag_7
, lag(ctarjeta_master_transacciones, 8) over ventana as ctarjeta_master_transacciones_lag_8
, ctarjeta_master_transacciones - lag(ctarjeta_master_transacciones, 1) over ventana as ctarjeta_master_transacciones_lag_delta_1
, ctarjeta_master_transacciones - lag(ctarjeta_master_transacciones, 2) over ventana as ctarjeta_master_transacciones_lag_delta_2
, ctarjeta_master_transacciones - lag(ctarjeta_master_transacciones, 3) over ventana as ctarjeta_master_transacciones_lag_delta_3
, ctarjeta_master_transacciones - lag(ctarjeta_master_transacciones, 4) over ventana as ctarjeta_master_transacciones_lag_delta_4
, ctarjeta_master_transacciones - lag(ctarjeta_master_transacciones, 5) over ventana as ctarjeta_master_transacciones_lag_delta_5
, ctarjeta_master_transacciones - lag(ctarjeta_master_transacciones, 6) over ventana as ctarjeta_master_transacciones_lag_delta_6
, ctarjeta_master_transacciones - lag(ctarjeta_master_transacciones, 7) over ventana as ctarjeta_master_transacciones_lag_delta_7
, ctarjeta_master_transacciones - lag(ctarjeta_master_transacciones, 8) over ventana as ctarjeta_master_transacciones_lag_delta_8
, avg(ctarjeta_master_transacciones) over ventana as ctarjeta_master_transacciones_media

, regr_slope(Visa_madelantodolares, cliente_antiguedad) over ventana as Visa_madelantodolares_slope
, lag(Visa_madelantodolares, 1) over ventana as Visa_madelantodolares_lag_1
, lag(Visa_madelantodolares, 2) over ventana as Visa_madelantodolares_lag_2
, lag(Visa_madelantodolares, 3) over ventana as Visa_madelantodolares_lag_3
, lag(Visa_madelantodolares, 4) over ventana as Visa_madelantodolares_lag_4
, lag(Visa_madelantodolares, 5) over ventana as Visa_madelantodolares_lag_5
, lag(Visa_madelantodolares, 6) over ventana as Visa_madelantodolares_lag_6
, lag(Visa_madelantodolares, 7) over ventana as Visa_madelantodolares_lag_7
, lag(Visa_madelantodolares, 8) over ventana as Visa_madelantodolares_lag_8
, Visa_madelantodolares - lag(Visa_madelantodolares, 1) over ventana as Visa_madelantodolares_lag_delta_1
, Visa_madelantodolares - lag(Visa_madelantodolares, 2) over ventana as Visa_madelantodolares_lag_delta_2
, Visa_madelantodolares - lag(Visa_madelantodolares, 3) over ventana as Visa_madelantodolares_lag_delta_3
, Visa_madelantodolares - lag(Visa_madelantodolares, 4) over ventana as Visa_madelantodolares_lag_delta_4
, Visa_madelantodolares - lag(Visa_madelantodolares, 5) over ventana as Visa_madelantodolares_lag_delta_5
, Visa_madelantodolares - lag(Visa_madelantodolares, 6) over ventana as Visa_madelantodolares_lag_delta_6
, Visa_madelantodolares - lag(Visa_madelantodolares, 7) over ventana as Visa_madelantodolares_lag_delta_7
, Visa_madelantodolares - lag(Visa_madelantodolares, 8) over ventana as Visa_madelantodolares_lag_delta_8
, avg(Visa_madelantodolares) over ventana as Visa_madelantodolares_media

, regr_slope(Visa_mlimitecompra, cliente_antiguedad) over ventana as Visa_mlimitecompra_slope
, lag(Visa_mlimitecompra, 1) over ventana as Visa_mlimitecompra_lag_1
, lag(Visa_mlimitecompra, 2) over ventana as Visa_mlimitecompra_lag_2
, lag(Visa_mlimitecompra, 3) over ventana as Visa_mlimitecompra_lag_3
, lag(Visa_mlimitecompra, 4) over ventana as Visa_mlimitecompra_lag_4
, lag(Visa_mlimitecompra, 5) over ventana as Visa_mlimitecompra_lag_5
, lag(Visa_mlimitecompra, 6) over ventana as Visa_mlimitecompra_lag_6
, lag(Visa_mlimitecompra, 7) over ventana as Visa_mlimitecompra_lag_7
, lag(Visa_mlimitecompra, 8) over ventana as Visa_mlimitecompra_lag_8
, Visa_mlimitecompra - lag(Visa_mlimitecompra, 1) over ventana as Visa_mlimitecompra_lag_delta_1
, Visa_mlimitecompra - lag(Visa_mlimitecompra, 2) over ventana as Visa_mlimitecompra_lag_delta_2
, Visa_mlimitecompra - lag(Visa_mlimitecompra, 3) over ventana as Visa_mlimitecompra_lag_delta_3
, Visa_mlimitecompra - lag(Visa_mlimitecompra, 4) over ventana as Visa_mlimitecompra_lag_delta_4
, Visa_mlimitecompra - lag(Visa_mlimitecompra, 5) over ventana as Visa_mlimitecompra_lag_delta_5
, Visa_mlimitecompra - lag(Visa_mlimitecompra, 6) over ventana as Visa_mlimitecompra_lag_delta_6
, Visa_mlimitecompra - lag(Visa_mlimitecompra, 7) over ventana as Visa_mlimitecompra_lag_delta_7
, Visa_mlimitecompra - lag(Visa_mlimitecompra, 8) over ventana as Visa_mlimitecompra_lag_delta_8
, avg(Visa_mlimitecompra) over ventana as Visa_mlimitecompra_media

, regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana as Visa_msaldopesos_slope
, lag(Visa_msaldopesos, 1) over ventana as Visa_msaldopesos_lag_1
, lag(Visa_msaldopesos, 2) over ventana as Visa_msaldopesos_lag_2
, lag(Visa_msaldopesos, 3) over ventana as Visa_msaldopesos_lag_3
, lag(Visa_msaldopesos, 4) over ventana as Visa_msaldopesos_lag_4
, lag(Visa_msaldopesos, 5) over ventana as Visa_msaldopesos_lag_5
, lag(Visa_msaldopesos, 6) over ventana as Visa_msaldopesos_lag_6
, lag(Visa_msaldopesos, 7) over ventana as Visa_msaldopesos_lag_7
, lag(Visa_msaldopesos, 8) over ventana as Visa_msaldopesos_lag_8
, Visa_msaldopesos - lag(Visa_msaldopesos, 1) over ventana as Visa_msaldopesos_lag_delta_1
, Visa_msaldopesos - lag(Visa_msaldopesos, 2) over ventana as Visa_msaldopesos_lag_delta_2
, Visa_msaldopesos - lag(Visa_msaldopesos, 3) over ventana as Visa_msaldopesos_lag_delta_3
, Visa_msaldopesos - lag(Visa_msaldopesos, 4) over ventana as Visa_msaldopesos_lag_delta_4
, Visa_msaldopesos - lag(Visa_msaldopesos, 5) over ventana as Visa_msaldopesos_lag_delta_5
, Visa_msaldopesos - lag(Visa_msaldopesos, 6) over ventana as Visa_msaldopesos_lag_delta_6
, Visa_msaldopesos - lag(Visa_msaldopesos, 7) over ventana as Visa_msaldopesos_lag_delta_7
, Visa_msaldopesos - lag(Visa_msaldopesos, 8) over ventana as Visa_msaldopesos_lag_delta_8
, avg(Visa_msaldopesos) over ventana as Visa_msaldopesos_media

, regr_slope(Visa_mpagospesos, cliente_antiguedad) over ventana as Visa_mpagospesos_slope
, lag(Visa_mpagospesos, 1) over ventana as Visa_mpagospesos_lag_1
, lag(Visa_mpagospesos, 2) over ventana as Visa_mpagospesos_lag_2
, lag(Visa_mpagospesos, 3) over ventana as Visa_mpagospesos_lag_3
, lag(Visa_mpagospesos, 4) over ventana as Visa_mpagospesos_lag_4
, lag(Visa_mpagospesos, 5) over ventana as Visa_mpagospesos_lag_5
, lag(Visa_mpagospesos, 6) over ventana as Visa_mpagospesos_lag_6
, lag(Visa_mpagospesos, 7) over ventana as Visa_mpagospesos_lag_7
, lag(Visa_mpagospesos, 8) over ventana as Visa_mpagospesos_lag_8
, Visa_mpagospesos - lag(Visa_mpagospesos, 1) over ventana as Visa_mpagospesos_lag_delta_1
, Visa_mpagospesos - lag(Visa_mpagospesos, 2) over ventana as Visa_mpagospesos_lag_delta_2
, Visa_mpagospesos - lag(Visa_mpagospesos, 3) over ventana as Visa_mpagospesos_lag_delta_3
, Visa_mpagospesos - lag(Visa_mpagospesos, 4) over ventana as Visa_mpagospesos_lag_delta_4
, Visa_mpagospesos - lag(Visa_mpagospesos, 5) over ventana as Visa_mpagospesos_lag_delta_5
, Visa_mpagospesos - lag(Visa_mpagospesos, 6) over ventana as Visa_mpagospesos_lag_delta_6
, Visa_mpagospesos - lag(Visa_mpagospesos, 7) over ventana as Visa_mpagospesos_lag_delta_7
, Visa_mpagospesos - lag(Visa_mpagospesos, 8) over ventana as Visa_mpagospesos_lag_delta_8
, avg(Visa_mpagospesos) over ventana as Visa_mpagospesos_media

, regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana as Visa_mpagominimo_slope
, lag(Visa_mpagominimo, 1) over ventana as Visa_mpagominimo_lag_1
, lag(Visa_mpagominimo, 2) over ventana as Visa_mpagominimo_lag_2
, lag(Visa_mpagominimo, 3) over ventana as Visa_mpagominimo_lag_3
, lag(Visa_mpagominimo, 4) over ventana as Visa_mpagominimo_lag_4
, lag(Visa_mpagominimo, 5) over ventana as Visa_mpagominimo_lag_5
, lag(Visa_mpagominimo, 6) over ventana as Visa_mpagominimo_lag_6
, lag(Visa_mpagominimo, 7) over ventana as Visa_mpagominimo_lag_7
, lag(Visa_mpagominimo, 8) over ventana as Visa_mpagominimo_lag_8
, Visa_mpagominimo - lag(Visa_mpagominimo, 1) over ventana as Visa_mpagominimo_lag_delta_1
, Visa_mpagominimo - lag(Visa_mpagominimo, 2) over ventana as Visa_mpagominimo_lag_delta_2
, Visa_mpagominimo - lag(Visa_mpagominimo, 3) over ventana as Visa_mpagominimo_lag_delta_3
, Visa_mpagominimo - lag(Visa_mpagominimo, 4) over ventana as Visa_mpagominimo_lag_delta_4
, Visa_mpagominimo - lag(Visa_mpagominimo, 5) over ventana as Visa_mpagominimo_lag_delta_5
, Visa_mpagominimo - lag(Visa_mpagominimo, 6) over ventana as Visa_mpagominimo_lag_delta_6
, Visa_mpagominimo - lag(Visa_mpagominimo, 7) over ventana as Visa_mpagominimo_lag_delta_7
, Visa_mpagominimo - lag(Visa_mpagominimo, 8) over ventana as Visa_mpagominimo_lag_delta_8
, avg(Visa_mpagominimo) over ventana as Visa_mpagominimo_media

, regr_slope(Visa_msaldodolares, cliente_antiguedad) over ventana as Visa_msaldodolares_slope
, lag(Visa_msaldodolares, 1) over ventana as Visa_msaldodolares_lag_1
, lag(Visa_msaldodolares, 2) over ventana as Visa_msaldodolares_lag_2
, lag(Visa_msaldodolares, 3) over ventana as Visa_msaldodolares_lag_3
, lag(Visa_msaldodolares, 4) over ventana as Visa_msaldodolares_lag_4
, lag(Visa_msaldodolares, 5) over ventana as Visa_msaldodolares_lag_5
, lag(Visa_msaldodolares, 6) over ventana as Visa_msaldodolares_lag_6
, lag(Visa_msaldodolares, 7) over ventana as Visa_msaldodolares_lag_7
, lag(Visa_msaldodolares, 8) over ventana as Visa_msaldodolares_lag_8
, Visa_msaldodolares - lag(Visa_msaldodolares, 1) over ventana as Visa_msaldodolares_lag_delta_1
, Visa_msaldodolares - lag(Visa_msaldodolares, 2) over ventana as Visa_msaldodolares_lag_delta_2
, Visa_msaldodolares - lag(Visa_msaldodolares, 3) over ventana as Visa_msaldodolares_lag_delta_3
, Visa_msaldodolares - lag(Visa_msaldodolares, 4) over ventana as Visa_msaldodolares_lag_delta_4
, Visa_msaldodolares - lag(Visa_msaldodolares, 5) over ventana as Visa_msaldodolares_lag_delta_5
, Visa_msaldodolares - lag(Visa_msaldodolares, 6) over ventana as Visa_msaldodolares_lag_delta_6
, Visa_msaldodolares - lag(Visa_msaldodolares, 7) over ventana as Visa_msaldodolares_lag_delta_7
, Visa_msaldodolares - lag(Visa_msaldodolares, 8) over ventana as Visa_msaldodolares_lag_delta_8
, avg(Visa_msaldodolares) over ventana as Visa_msaldodolares_media

, regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana as Visa_msaldototal_slope
, lag(Visa_msaldototal, 1) over ventana as Visa_msaldototal_lag_1
, lag(Visa_msaldototal, 2) over ventana as Visa_msaldototal_lag_2
, lag(Visa_msaldototal, 3) over ventana as Visa_msaldototal_lag_3
, lag(Visa_msaldototal, 4) over ventana as Visa_msaldototal_lag_4
, lag(Visa_msaldototal, 5) over ventana as Visa_msaldototal_lag_5
, lag(Visa_msaldototal, 6) over ventana as Visa_msaldototal_lag_6
, lag(Visa_msaldototal, 7) over ventana as Visa_msaldototal_lag_7
, lag(Visa_msaldototal, 8) over ventana as Visa_msaldototal_lag_8
, Visa_msaldototal - lag(Visa_msaldototal, 1) over ventana as Visa_msaldototal_lag_delta_1
, Visa_msaldototal - lag(Visa_msaldototal, 2) over ventana as Visa_msaldototal_lag_delta_2
, Visa_msaldototal - lag(Visa_msaldototal, 3) over ventana as Visa_msaldototal_lag_delta_3
, Visa_msaldototal - lag(Visa_msaldototal, 4) over ventana as Visa_msaldototal_lag_delta_4
, Visa_msaldototal - lag(Visa_msaldototal, 5) over ventana as Visa_msaldototal_lag_delta_5
, Visa_msaldototal - lag(Visa_msaldototal, 6) over ventana as Visa_msaldototal_lag_delta_6
, Visa_msaldototal - lag(Visa_msaldototal, 7) over ventana as Visa_msaldototal_lag_delta_7
, Visa_msaldototal - lag(Visa_msaldototal, 8) over ventana as Visa_msaldototal_lag_delta_8
, avg(Visa_msaldototal) over ventana as Visa_msaldototal_media

, regr_slope(Visa_cconsumos, cliente_antiguedad) over ventana as Visa_cconsumos_slope
, lag(Visa_cconsumos, 1) over ventana as Visa_cconsumos_lag_1
, lag(Visa_cconsumos, 2) over ventana as Visa_cconsumos_lag_2
, lag(Visa_cconsumos, 3) over ventana as Visa_cconsumos_lag_3
, lag(Visa_cconsumos, 4) over ventana as Visa_cconsumos_lag_4
, lag(Visa_cconsumos, 5) over ventana as Visa_cconsumos_lag_5
, lag(Visa_cconsumos, 6) over ventana as Visa_cconsumos_lag_6
, lag(Visa_cconsumos, 7) over ventana as Visa_cconsumos_lag_7
, lag(Visa_cconsumos, 8) over ventana as Visa_cconsumos_lag_8
, Visa_cconsumos - lag(Visa_cconsumos, 1) over ventana as Visa_cconsumos_lag_delta_1
, Visa_cconsumos - lag(Visa_cconsumos, 2) over ventana as Visa_cconsumos_lag_delta_2
, Visa_cconsumos - lag(Visa_cconsumos, 3) over ventana as Visa_cconsumos_lag_delta_3
, Visa_cconsumos - lag(Visa_cconsumos, 4) over ventana as Visa_cconsumos_lag_delta_4
, Visa_cconsumos - lag(Visa_cconsumos, 5) over ventana as Visa_cconsumos_lag_delta_5
, Visa_cconsumos - lag(Visa_cconsumos, 6) over ventana as Visa_cconsumos_lag_delta_6
, Visa_cconsumos - lag(Visa_cconsumos, 7) over ventana as Visa_cconsumos_lag_delta_7
, Visa_cconsumos - lag(Visa_cconsumos, 8) over ventana as Visa_cconsumos_lag_delta_8
, avg(Visa_cconsumos) over ventana as Visa_cconsumos_media

, regr_slope(Visa_mconsumospesos, cliente_antiguedad) over ventana as Visa_mconsumospesos_slope
, lag(Visa_mconsumospesos, 1) over ventana as Visa_mconsumospesos_lag_1
, lag(Visa_mconsumospesos, 2) over ventana as Visa_mconsumospesos_lag_2
, lag(Visa_mconsumospesos, 3) over ventana as Visa_mconsumospesos_lag_3
, lag(Visa_mconsumospesos, 4) over ventana as Visa_mconsumospesos_lag_4
, lag(Visa_mconsumospesos, 5) over ventana as Visa_mconsumospesos_lag_5
, lag(Visa_mconsumospesos, 6) over ventana as Visa_mconsumospesos_lag_6
, lag(Visa_mconsumospesos, 7) over ventana as Visa_mconsumospesos_lag_7
, lag(Visa_mconsumospesos, 8) over ventana as Visa_mconsumospesos_lag_8
, Visa_mconsumospesos - lag(Visa_mconsumospesos, 1) over ventana as Visa_mconsumospesos_lag_delta_1
, Visa_mconsumospesos - lag(Visa_mconsumospesos, 2) over ventana as Visa_mconsumospesos_lag_delta_2
, Visa_mconsumospesos - lag(Visa_mconsumospesos, 3) over ventana as Visa_mconsumospesos_lag_delta_3
, Visa_mconsumospesos - lag(Visa_mconsumospesos, 4) over ventana as Visa_mconsumospesos_lag_delta_4
, Visa_mconsumospesos - lag(Visa_mconsumospesos, 5) over ventana as Visa_mconsumospesos_lag_delta_5
, Visa_mconsumospesos - lag(Visa_mconsumospesos, 6) over ventana as Visa_mconsumospesos_lag_delta_6
, Visa_mconsumospesos - lag(Visa_mconsumospesos, 7) over ventana as Visa_mconsumospesos_lag_delta_7
, Visa_mconsumospesos - lag(Visa_mconsumospesos, 8) over ventana as Visa_mconsumospesos_lag_delta_8
, avg(Visa_mconsumospesos) over ventana as Visa_mconsumospesos_media

, regr_slope(Master_madelantodolares, cliente_antiguedad) over ventana as Master_madelantodolares_slope
, lag(Master_madelantodolares, 1) over ventana as Master_madelantodolares_lag_1
, lag(Master_madelantodolares, 2) over ventana as Master_madelantodolares_lag_2
, lag(Master_madelantodolares, 3) over ventana as Master_madelantodolares_lag_3
, lag(Master_madelantodolares, 4) over ventana as Master_madelantodolares_lag_4
, lag(Master_madelantodolares, 5) over ventana as Master_madelantodolares_lag_5
, lag(Master_madelantodolares, 6) over ventana as Master_madelantodolares_lag_6
, lag(Master_madelantodolares, 7) over ventana as Master_madelantodolares_lag_7
, lag(Master_madelantodolares, 8) over ventana as Master_madelantodolares_lag_8
, Master_madelantodolares - lag(Master_madelantodolares, 1) over ventana as Master_madelantodolares_lag_delta_1
, Master_madelantodolares - lag(Master_madelantodolares, 2) over ventana as Master_madelantodolares_lag_delta_2
, Master_madelantodolares - lag(Master_madelantodolares, 3) over ventana as Master_madelantodolares_lag_delta_3
, Master_madelantodolares - lag(Master_madelantodolares, 4) over ventana as Master_madelantodolares_lag_delta_4
, Master_madelantodolares - lag(Master_madelantodolares, 5) over ventana as Master_madelantodolares_lag_delta_5
, Master_madelantodolares - lag(Master_madelantodolares, 6) over ventana as Master_madelantodolares_lag_delta_6
, Master_madelantodolares - lag(Master_madelantodolares, 7) over ventana as Master_madelantodolares_lag_delta_7
, Master_madelantodolares - lag(Master_madelantodolares, 8) over ventana as Master_madelantodolares_lag_delta_8
, avg(Master_madelantodolares) over ventana as Master_madelantodolares_media

, regr_slope(Master_mlimitecompra, cliente_antiguedad) over ventana as Master_mlimitecompra_slope
, lag(Master_mlimitecompra, 1) over ventana as Master_mlimitecompra_lag_1
, lag(Master_mlimitecompra, 2) over ventana as Master_mlimitecompra_lag_2
, lag(Master_mlimitecompra, 3) over ventana as Master_mlimitecompra_lag_3
, lag(Master_mlimitecompra, 4) over ventana as Master_mlimitecompra_lag_4
, lag(Master_mlimitecompra, 5) over ventana as Master_mlimitecompra_lag_5
, lag(Master_mlimitecompra, 6) over ventana as Master_mlimitecompra_lag_6
, lag(Master_mlimitecompra, 7) over ventana as Master_mlimitecompra_lag_7
, lag(Master_mlimitecompra, 8) over ventana as Master_mlimitecompra_lag_8
, Master_mlimitecompra - lag(Master_mlimitecompra, 1) over ventana as Master_mlimitecompra_lag_delta_1
, Master_mlimitecompra - lag(Master_mlimitecompra, 2) over ventana as Master_mlimitecompra_lag_delta_2
, Master_mlimitecompra - lag(Master_mlimitecompra, 3) over ventana as Master_mlimitecompra_lag_delta_3
, Master_mlimitecompra - lag(Master_mlimitecompra, 4) over ventana as Master_mlimitecompra_lag_delta_4
, Master_mlimitecompra - lag(Master_mlimitecompra, 5) over ventana as Master_mlimitecompra_lag_delta_5
, Master_mlimitecompra - lag(Master_mlimitecompra, 6) over ventana as Master_mlimitecompra_lag_delta_6
, Master_mlimitecompra - lag(Master_mlimitecompra, 7) over ventana as Master_mlimitecompra_lag_delta_7
, Master_mlimitecompra - lag(Master_mlimitecompra, 8) over ventana as Master_mlimitecompra_lag_delta_8
, avg(Master_mlimitecompra) over ventana as Master_mlimitecompra_media

, regr_slope(Master_msaldopesos, cliente_antiguedad) over ventana as Master_msaldopesos_slope
, lag(Master_msaldopesos, 1) over ventana as Master_msaldopesos_lag_1
, lag(Master_msaldopesos, 2) over ventana as Master_msaldopesos_lag_2
, lag(Master_msaldopesos, 3) over ventana as Master_msaldopesos_lag_3
, lag(Master_msaldopesos, 4) over ventana as Master_msaldopesos_lag_4
, lag(Master_msaldopesos, 5) over ventana as Master_msaldopesos_lag_5
, lag(Master_msaldopesos, 6) over ventana as Master_msaldopesos_lag_6
, lag(Master_msaldopesos, 7) over ventana as Master_msaldopesos_lag_7
, lag(Master_msaldopesos, 8) over ventana as Master_msaldopesos_lag_8
, Master_msaldopesos - lag(Master_msaldopesos, 1) over ventana as Master_msaldopesos_lag_delta_1
, Master_msaldopesos - lag(Master_msaldopesos, 2) over ventana as Master_msaldopesos_lag_delta_2
, Master_msaldopesos - lag(Master_msaldopesos, 3) over ventana as Master_msaldopesos_lag_delta_3
, Master_msaldopesos - lag(Master_msaldopesos, 4) over ventana as Master_msaldopesos_lag_delta_4
, Master_msaldopesos - lag(Master_msaldopesos, 5) over ventana as Master_msaldopesos_lag_delta_5
, Master_msaldopesos - lag(Master_msaldopesos, 6) over ventana as Master_msaldopesos_lag_delta_6
, Master_msaldopesos - lag(Master_msaldopesos, 7) over ventana as Master_msaldopesos_lag_delta_7
, Master_msaldopesos - lag(Master_msaldopesos, 8) over ventana as Master_msaldopesos_lag_delta_8
, avg(Master_msaldopesos) over ventana as Master_msaldopesos_media

, regr_slope(Master_mpagospesos, cliente_antiguedad) over ventana as Master_mpagospesos_slope
, lag(Master_mpagospesos, 1) over ventana as Master_mpagospesos_lag_1
, lag(Master_mpagospesos, 2) over ventana as Master_mpagospesos_lag_2
, lag(Master_mpagospesos, 3) over ventana as Master_mpagospesos_lag_3
, lag(Master_mpagospesos, 4) over ventana as Master_mpagospesos_lag_4
, lag(Master_mpagospesos, 5) over ventana as Master_mpagospesos_lag_5
, lag(Master_mpagospesos, 6) over ventana as Master_mpagospesos_lag_6
, lag(Master_mpagospesos, 7) over ventana as Master_mpagospesos_lag_7
, lag(Master_mpagospesos, 8) over ventana as Master_mpagospesos_lag_8
, Master_mpagospesos - lag(Master_mpagospesos, 1) over ventana as Master_mpagospesos_lag_delta_1
, Master_mpagospesos - lag(Master_mpagospesos, 2) over ventana as Master_mpagospesos_lag_delta_2
, Master_mpagospesos - lag(Master_mpagospesos, 3) over ventana as Master_mpagospesos_lag_delta_3
, Master_mpagospesos - lag(Master_mpagospesos, 4) over ventana as Master_mpagospesos_lag_delta_4
, Master_mpagospesos - lag(Master_mpagospesos, 5) over ventana as Master_mpagospesos_lag_delta_5
, Master_mpagospesos - lag(Master_mpagospesos, 6) over ventana as Master_mpagospesos_lag_delta_6
, Master_mpagospesos - lag(Master_mpagospesos, 7) over ventana as Master_mpagospesos_lag_delta_7
, Master_mpagospesos - lag(Master_mpagospesos, 8) over ventana as Master_mpagospesos_lag_delta_8
, avg(Master_mpagospesos) over ventana as Master_mpagospesos_media

, regr_slope(Master_mpagominimo, cliente_antiguedad) over ventana as Master_mpagominimo_slope
, lag(Master_mpagominimo, 1) over ventana as Master_mpagominimo_lag_1
, lag(Master_mpagominimo, 2) over ventana as Master_mpagominimo_lag_2
, lag(Master_mpagominimo, 3) over ventana as Master_mpagominimo_lag_3
, lag(Master_mpagominimo, 4) over ventana as Master_mpagominimo_lag_4
, lag(Master_mpagominimo, 5) over ventana as Master_mpagominimo_lag_5
, lag(Master_mpagominimo, 6) over ventana as Master_mpagominimo_lag_6
, lag(Master_mpagominimo, 7) over ventana as Master_mpagominimo_lag_7
, lag(Master_mpagominimo, 8) over ventana as Master_mpagominimo_lag_8
, Master_mpagominimo - lag(Master_mpagominimo, 1) over ventana as Master_mpagominimo_lag_delta_1
, Master_mpagominimo - lag(Master_mpagominimo, 2) over ventana as Master_mpagominimo_lag_delta_2
, Master_mpagominimo - lag(Master_mpagominimo, 3) over ventana as Master_mpagominimo_lag_delta_3
, Master_mpagominimo - lag(Master_mpagominimo, 4) over ventana as Master_mpagominimo_lag_delta_4
, Master_mpagominimo - lag(Master_mpagominimo, 5) over ventana as Master_mpagominimo_lag_delta_5
, Master_mpagominimo - lag(Master_mpagominimo, 6) over ventana as Master_mpagominimo_lag_delta_6
, Master_mpagominimo - lag(Master_mpagominimo, 7) over ventana as Master_mpagominimo_lag_delta_7
, Master_mpagominimo - lag(Master_mpagominimo, 8) over ventana as Master_mpagominimo_lag_delta_8
, avg(Master_mpagominimo) over ventana as Master_mpagominimo_media

, regr_slope(Master_msaldodolares, cliente_antiguedad) over ventana as Master_msaldodolares_slope
, lag(Master_msaldodolares, 1) over ventana as Master_msaldodolares_lag_1
, lag(Master_msaldodolares, 2) over ventana as Master_msaldodolares_lag_2
, lag(Master_msaldodolares, 3) over ventana as Master_msaldodolares_lag_3
, lag(Master_msaldodolares, 4) over ventana as Master_msaldodolares_lag_4
, lag(Master_msaldodolares, 5) over ventana as Master_msaldodolares_lag_5
, lag(Master_msaldodolares, 6) over ventana as Master_msaldodolares_lag_6
, lag(Master_msaldodolares, 7) over ventana as Master_msaldodolares_lag_7
, lag(Master_msaldodolares, 8) over ventana as Master_msaldodolares_lag_8
, Master_msaldodolares - lag(Master_msaldodolares, 1) over ventana as Master_msaldodolares_lag_delta_1
, Master_msaldodolares - lag(Master_msaldodolares, 2) over ventana as Master_msaldodolares_lag_delta_2
, Master_msaldodolares - lag(Master_msaldodolares, 3) over ventana as Master_msaldodolares_lag_delta_3
, Master_msaldodolares - lag(Master_msaldodolares, 4) over ventana as Master_msaldodolares_lag_delta_4
, Master_msaldodolares - lag(Master_msaldodolares, 5) over ventana as Master_msaldodolares_lag_delta_5
, Master_msaldodolares - lag(Master_msaldodolares, 6) over ventana as Master_msaldodolares_lag_delta_6
, Master_msaldodolares - lag(Master_msaldodolares, 7) over ventana as Master_msaldodolares_lag_delta_7
, Master_msaldodolares - lag(Master_msaldodolares, 8) over ventana as Master_msaldodolares_lag_delta_8
, avg(Master_msaldodolares) over ventana as Master_msaldodolares_media

, regr_slope(Master_msaldototal, cliente_antiguedad) over ventana as Master_msaldototal_slope
, lag(Master_msaldototal, 1) over ventana as Master_msaldototal_lag_1
, lag(Master_msaldototal, 2) over ventana as Master_msaldototal_lag_2
, lag(Master_msaldototal, 3) over ventana as Master_msaldototal_lag_3
, lag(Master_msaldototal, 4) over ventana as Master_msaldototal_lag_4
, lag(Master_msaldototal, 5) over ventana as Master_msaldototal_lag_5
, lag(Master_msaldototal, 6) over ventana as Master_msaldototal_lag_6
, lag(Master_msaldototal, 7) over ventana as Master_msaldototal_lag_7
, lag(Master_msaldototal, 8) over ventana as Master_msaldototal_lag_8
, Master_msaldototal - lag(Master_msaldototal, 1) over ventana as Master_msaldototal_lag_delta_1
, Master_msaldototal - lag(Master_msaldototal, 2) over ventana as Master_msaldototal_lag_delta_2
, Master_msaldototal - lag(Master_msaldototal, 3) over ventana as Master_msaldototal_lag_delta_3
, Master_msaldototal - lag(Master_msaldototal, 4) over ventana as Master_msaldototal_lag_delta_4
, Master_msaldototal - lag(Master_msaldototal, 5) over ventana as Master_msaldototal_lag_delta_5
, Master_msaldototal - lag(Master_msaldototal, 6) over ventana as Master_msaldototal_lag_delta_6
, Master_msaldototal - lag(Master_msaldototal, 7) over ventana as Master_msaldototal_lag_delta_7
, Master_msaldototal - lag(Master_msaldototal, 8) over ventana as Master_msaldototal_lag_delta_8
, avg(Master_msaldototal) over ventana as Master_msaldototal_media

, regr_slope(Master_mconsumospesos, cliente_antiguedad) over ventana as Master_mconsumospesos_slope
, lag(Master_mconsumospesos, 1) over ventana as Master_mconsumospesos_lag_1
, lag(Master_mconsumospesos, 2) over ventana as Master_mconsumospesos_lag_2
, lag(Master_mconsumospesos, 3) over ventana as Master_mconsumospesos_lag_3
, lag(Master_mconsumospesos, 4) over ventana as Master_mconsumospesos_lag_4
, lag(Master_mconsumospesos, 5) over ventana as Master_mconsumospesos_lag_5
, lag(Master_mconsumospesos, 6) over ventana as Master_mconsumospesos_lag_6
, lag(Master_mconsumospesos, 7) over ventana as Master_mconsumospesos_lag_7
, lag(Master_mconsumospesos, 8) over ventana as Master_mconsumospesos_lag_8
, Master_mconsumospesos - lag(Master_mconsumospesos, 1) over ventana as Master_mconsumospesos_lag_delta_1
, Master_mconsumospesos - lag(Master_mconsumospesos, 2) over ventana as Master_mconsumospesos_lag_delta_2
, Master_mconsumospesos - lag(Master_mconsumospesos, 3) over ventana as Master_mconsumospesos_lag_delta_3
, Master_mconsumospesos - lag(Master_mconsumospesos, 4) over ventana as Master_mconsumospesos_lag_delta_4
, Master_mconsumospesos - lag(Master_mconsumospesos, 5) over ventana as Master_mconsumospesos_lag_delta_5
, Master_mconsumospesos - lag(Master_mconsumospesos, 6) over ventana as Master_mconsumospesos_lag_delta_6
, Master_mconsumospesos - lag(Master_mconsumospesos, 7) over ventana as Master_mconsumospesos_lag_delta_7
, Master_mconsumospesos - lag(Master_mconsumospesos, 8) over ventana as Master_mconsumospesos_lag_delta_8
, avg(Master_mconsumospesos) over ventana as Master_mconsumospesos_media

, regr_slope(Master_cconsumos, cliente_antiguedad) over ventana as Master_cconsumos_slope
, lag(Master_cconsumos, 1) over ventana as Master_cconsumos_lag_1
, lag(Master_cconsumos, 2) over ventana as Master_cconsumos_lag_2
, lag(Master_cconsumos, 3) over ventana as Master_cconsumos_lag_3
, lag(Master_cconsumos, 4) over ventana as Master_cconsumos_lag_4
, lag(Master_cconsumos, 5) over ventana as Master_cconsumos_lag_5
, lag(Master_cconsumos, 6) over ventana as Master_cconsumos_lag_6
, lag(Master_cconsumos, 7) over ventana as Master_cconsumos_lag_7
, lag(Master_cconsumos, 8) over ventana as Master_cconsumos_lag_8
, Master_cconsumos - lag(Master_cconsumos, 1) over ventana as Master_cconsumos_lag_delta_1
, Master_cconsumos - lag(Master_cconsumos, 2) over ventana as Master_cconsumos_lag_delta_2
, Master_cconsumos - lag(Master_cconsumos, 3) over ventana as Master_cconsumos_lag_delta_3
, Master_cconsumos - lag(Master_cconsumos, 4) over ventana as Master_cconsumos_lag_delta_4
, Master_cconsumos - lag(Master_cconsumos, 5) over ventana as Master_cconsumos_lag_delta_5
, Master_cconsumos - lag(Master_cconsumos, 6) over ventana as Master_cconsumos_lag_delta_6
, Master_cconsumos - lag(Master_cconsumos, 7) over ventana as Master_cconsumos_lag_delta_7
, Master_cconsumos - lag(Master_cconsumos, 8) over ventana as Master_cconsumos_lag_delta_8
, avg(Master_cconsumos) over ventana as Master_cconsumos_media

FROM competencia_03
WHERE foto_mes != 202006
WINDOW ventana AS (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 8 PRECEDING AND CURRENT ROW)


CPU times: total: 21min 31s
Wall time: 14min 4s


,Success


## Escribo el archivo

In [9]:
%%time
%%sql
copy resultado to 'c:/databases/datasets/competencia_03_lag_delta_8.csv.gz' (FORMAT CSV, HEADER)

CPU times: total: 41min 4s
Wall time: 18min 46s


,Success
